In [1]:
#%pip install pandas, openpyxl, matplotlib,  python-docx beautifulsoup4 requests html5lib

In [1]:
import pandas as pd
import openpyxl
import requests
from bs4 import BeautifulSoup
import docx
import os
from win32com import client
import time
#import html5lib


In [2]:
filename = 'Abstracts_BonnBrain2024.xlsx'
#filename = 'Abstracts_from_web_site.csv'

df = pd.read_excel(filename)
pd.set_option('display.max_rows', 20)

In [3]:
nAbstracts = len(df)
nAbstracts

107

In [4]:
df

,Abstract #,Title,Author,Last author,Topic,Reviewer,talk suggestion? (Yes/No),comment,Willing to give a talk,Unnamed: 9,First Author Last Name
0,64.0,The impact of social categorization on the rep...,Ahmed,Omar Ahmed,Computational / Modeling,Jan Gründemann,x,NaN,NaN,NaN,Puvvada
1,69.0,Entropic Force or a Homeostatic Mechanism Can ...,Altrogge,Raoul-Martin Memmesheimer,Computational / Modeling,Martin Fuhrmann,NaN,NaN,NaN,NaN,Lenzi
2,99.0,Local migration and proliferation of resident ...,Antony,Martin Fuhrmann,Glia,Tobias Rose,NaN,NaN,NaN,NaN,Tran
3,29.0,Circuit Mechanisms of Latent Learning in the D...,Araujo,Tobias Rose,Circuits,Ilona Grunwald Kadow,NaN,NaN,NaN,NaN,Chikkankod
4,17.0,Contribution of Leptin Signaling to the Sex- a...,Awad,Anne Petzold,Circuits,Ilona Grunwald Kadow,yes,NaN,NaN,NaN,Castro-Gomez
...,...,...,...,...,...,...,...,...,...,...,...
102,58.0,Predictive Power of Behavior: Robust Low-Dimen...,Verzelli,Tatjana Tchumatchenko,Computational / Modeling,Sabine Krabbe,yes,NaN,NaN,NaN,Guha
103,3.0,A body-brain axis between fatbody and brain re...,Wang,Yujie Wang,Cellular,Jason Kerr,NaN,NaN,NaN,NaN,Rachid
104,11.0,Neural substrates in the postpartum brain for ...,Yang,Silvana Valtcheva,Circuits,Sabine Krabbe,yes,NaN,NaN,NaN,Kelly
105,102.0,An automated behavioral platform for multisens...,Yousefi,Fatemeh Yousefi,Technology,Jason Kerr,NaN,NaN,NaN,NaN,Patel


#### Get URL links to all abstracts

In [5]:
wb = openpyxl.load_workbook(filename)
ws = wb['All_Abstracts']

In [18]:
def get_all_info_from_site(html):
    soup = document = BeautifulSoup(html, "html.parser")

    
    # Get title of abstract
    item = soup.find("h1", class_="h1_abstracts")
    title = item.get_text().lower().capitalize()

    # Get abstract text
    items = soup.find_all("p", class_="p_abstract")

    for item in items:
        abstract = item.get_text()[1:]

    #print('Abstract: ', abstract)

    # Get author names and number indicating affiliation
    item = soup.find("p", class_="p_authors")
    author_list_raw = item.get_text().split(',')

    author_list = list()
    author_institute_affil = list()
    for author_name_raw in author_list_raw:
        author_list.append(author_name_raw.strip()[:-1])
        author_institute_affil.append(author_name_raw.strip()[-1])

    # Get names of institutes the authors are affiliated with
    item = soup.find("p", class_="p_institutes")
    institute_list_raw = item.get_text().split('\n')

    institute_list = list()
    for institute_name_raw in institute_list_raw[1:-1]:
        institute_list.append(institute_name_raw.strip()[2:])

    return title, abstract, author_list, author_institute_affil, institute_list

In [19]:
def write_to_docx_file(document):

    # Add a paragraph to the document
    p = document.add_paragraph()
    #document.add_heading(title, level=1)
    title_abstract = p.add_run(title)
    # Add some formatting to the run
    title_abstract.bold = True
    #title_abstract.italic = True
    title_abstract.font.name = 'Arial'
    title_abstract.font.size = docx.shared.Pt(16)

    # Add more text to the same paragraph
    p = document.add_paragraph()

    # Add a run to the paragraph
    for iAuthor, author_name in enumerate(author_list):
        if iAuthor > 0:
            p.add_run(', ')
        if iAuthor == 0:
            first_author = p.add_run(author_name)
            first_author.bold = True
        else:
            p.add_run(author_name)

        super_text = p.add_run(author_institute_affil[iAuthor])
        super_text.font.superscript = True

    #p = document.add_paragraph()

    for iInstitute, institute_name in enumerate(institute_list):

        p = document.add_paragraph()

        super_text = p.add_run(str(iInstitute+1))
        super_text.font.superscript = True

        run = p.add_run(institute_name)
        run.font.size = docx.shared.Pt(10)
        # Add some formatting to the paragraph
        p.paragraph_format.line_spacing = 1.2
        p.paragraph_format.space_after = 4

    p.paragraph_format.line_spacing = 2

    p = document.add_paragraph()
    #p.paragraph_format.line_spacing = 1.0

    p.add_run(abstract)

    # Adding a page break 
    document.add_page_break() 

    document.save('test.docx')

In [22]:
document = docx.Document()

for iAbstract in range(3):

    print(iAbstract)

    try:
        # This will fail if there is no hyperlink to target
        url = ws.cell(row=iAbstract+2, column=2).hyperlink.target
    except Exception as e:
        print(e, '. Likely no abstract available yet.')
    r = requests.get(url)
    assert r.status_code == 200
    html = r.text

    title, abstract, author_list, author_institute_affil, institute_list = get_all_info_from_site(html = html)

    print(title)
    write_to_docx_file(document=document)

word = client.DispatchEx("Word.Application") # Using DispatchEx for an entirely new Word instance
word.Visible = True # Added this in here so you can see what I'm talking about with the movement of the dispatch and Quit lines. 
file_type = 'docx'
folder = '.'
out_folder = folder + "\\PDF"
for files in os.listdir("."):
    if files.endswith("test.docx"):
        out_name = files.replace(file_type, "pdf")
        in_file = os.path.abspath(folder + "\\" + files)
        out_file = os.path.abspath(out_folder + "\\" + out_name)
        doc = word.Documents.Open(in_file)
        print('Exporting', out_file)
        time.sleep(10)
        #doc.SaveAs(out_file, FileFormat=17)
        doc.Close()



0
Abstract:  Social categorisation is associated with attitudes towards social groups that include ingroup bias, stereotyping or dehumanisation, which in turn facilitate behaviours like social support, discrimination or aggression. This categorisation influences even early stages of social interactions, such as face perception: ingroup faces evoke di;erent neural responses than outgroup faces, whether the groups are racial or arbitrary.
We hypothesised that revealing the category (here: study fields) of students would lead fellow students to categorise them into in- vs. out-group members. Categorisation should be reflected in changes of pairwise similarity between these individuals. In a withinsubject design, 20 participants were shown the faces of eight unknown students. Participants rated the pairwise similarity between these students based on facial features, social life, and personality. After the students’ study fields were revealed, participants repeated the similarity ratings.
W

AttributeError: Open.Close

In [108]:
from docx import Document
from docx.shared import Inches

document = Document()

document.add_heading('Document Title', 0)

p = document.add_paragraph('A plain paragraph having some ')
p.add_run('bold').bold = True
p.add_run(' and some ')
p.add_run('italic.').italic = True

document.add_heading('Heading, level 1', level=1)
document.add_paragraph('Intense quote', style='Intense Quote')

document.add_paragraph(
    'first item in unordered list', style='List Bullet'
)
document.add_paragraph(
    'first item in ordered list', style='List Number'
)

#document.add_picture('monty-truth.png', width=Inches(1.25))

records = (
    (3, '101', 'Spam'),
    (7, '422', 'Eggs'),
    (4, '631', 'Spam, spam, eggs, and spam')
)

table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
for qty, id, desc in records:
    row_cells = table.add_row().cells
    row_cells[0].text = str(qty)
    row_cells[1].text = id
    row_cells[2].text = desc

document.add_page_break()

document.save('demo.docx')

In [51]:
#soup = BeautifulSoup(r.content, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <!--[if IE]>
	<meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=edge" /><![endif]-->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Modulation of visual processing by the Melanocortin system in Larval Zebrafish – BonnBrain³ Meeting
  </title>
  <meta content="max-image-preview:large" name="robots">
   <link href="//maxcdn.bootstrapcdn.com" rel="dns-prefetch"/>
   <link href="//fonts.googleapis.com" rel="dns-prefetch"/>
   <link href="https://bonnbrain.de/feed/" rel="alternate" title="BonnBrain³ Meeting » Feed" type="application/rss+xml"/>
   <link href="https://bonnbrain.de/comments/feed/" rel="alternate" title="BonnBrain³ Meeting » Comments Feed" type="application/rss+xml"/>
   <script type="text/javascript">
    /* <![CDATA[ */
window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/14.0.0\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.o

In [29]:
results = soup.find(id="Title")
print(results.prettify())


AttributeError: 'NoneType' object has no attribute 'prettify'

In [16]:
quotes = []

table = soup.find('div', attrs = {'id':'all_quotes'}) 

for row in table.findAll('div', attrs = {'class':'col-6 col-lg-3 text-center margin-30px-bottom sm-margin-30px-top'}):
    quote = {}
    quote['theme'] = row.h5.text
    quote['url'] = row.a['href']
    quote['img'] = row.img['src']
    #quote['lines'] = row.img['alt'].split(&quot; &quot;)[0]
    #quote['author'] = row.img['alt'].split(&quot; &quot;)[1]
    quotes.append(quote)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [23]:
r.url

'https://www.passiton.com/inspirational-quotes&quot'

In [20]:
import requests
from bs4 import BeautifulSoup
import csv
 
URL = "http://www.values.com/inspirational-quotes&quot"
r = requests.get(URL)
 
soup = BeautifulSoup(r.content, 'html5lib')
 
quotes=[]  # a list to store quotes
 
table = soup.find('div', attrs = {'id':'all_quotes'}) 
 
for row in table.findAll('div',
                         attrs = {'class':'col-6 col-lg-3 text-center margin-30px-bottom sm-margin-30px-top'}):
    quote = {}
    quote['theme'] = row.h5.text
    quote['url'] = row.a['href']
    quote['img'] = row.img['src']
    #quote['lines'] = row.img['alt'].split(&quot; &quot;)[0]
    #quote['author'] = row.img['alt'].split(&quot; &quot;)[1]
    quotes.append(quote)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [21]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" dir="ltr" lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
  <title>
   The page you were looking for doesn't exist | The Foundation for a Better Life
  </title>
  <meta content="Error 404" name="description"/>
  <meta content="pass, it, on, passiton, values, kindness" name="keywords"/>
  <meta content="The Foundation for a Better Life" name="twitter:site_name"/>
  <meta content="@passiton_values" name="twitter:site"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="Thank you for visiting." name="twitter:description"/>
  <meta content="https://www.passiton.com/passiton_fbl.jpg" name="twitter:image"/>
  <meta content="https://www.passiton.com/404" property="og:url"/>
  <meta content="The Foundation for a Better Life" property="og:site_nam

In [3]:
df = df_temp.head(120)

In [4]:
pd.set_option('display.max_rows', 500)
df['allauthors[]']

0      Shahzad|Munir|Institute for Anatomy and Cell B...
1      Ram Reddy|Chandupatla|DZNE, Bonn|Regina|Feeder...
2      Senthilvelrajan|Kaniyappan|DZNE, Bonn|Katharin...
3      Ibtihel|Dhaya|Univ. Bordeaux, INCIA, UMR 5287,...
4      Aleksandra|Shiian|LIMES (Life and Medical Scie...
5      Sonja|Meiser|Computational Neuroscience, Depar...
6      Stephanie|Lohmann|German Center for Neurodegen...
7                         Xun|Tu|MPFI|Ryohei|Yasuda|MPFI
8      Jens F.|Schweihoff|Medical Faculty, University...
9      Anne|Quatraccioni|Institute of Neuropathology,...
10     Moritz Wolfram|Leweke|Institute of Experimenta...
11     Lea Jessica|Flitsch|Institute of Reconstructiv...
12     Alberto|Pauletti|Institute of Cellular Neurosc...
13     Kevin|Luxem|DZNE Bonn|Falko|Fuhrmann|DZNE Bonn...
14     Peter|Bedner|University of Bonn, Institute of ...
15     Kirsten|Bohmbach|Institute of Cellular Neurosc...
16     Sebastian|Dupraz|Axonal Growth and Regeneratio...
17     Eva Maria|Schönhense|Ins

In [47]:
abstract_texts_raw = df['Content']
df_abstracts = df['Content']

In [ ]:
# Figure how to write to file what you get when you print the abstracts (that is, get them with the html codes interpreted)

In [48]:
df_abstracts 

0      <p>Dopamine is an important modulatory neurotr...
1      <p>The repeat domain of Tau protein with the p...
2      <p>Tau aggregation into amyloid-like fibers ba...
3      <p><span lang="en-US">Magnetic Resonance Imagi...
4      <p>The balance between food consumption and ut...
5      Touch mechanoreceptors (T cells) of the leech ...
6      <p>In synucleinopathies such as Parkinson’s di...
7      <p>The  insulin superfamily of peptides are we...
8      <p>In general, animal behavior can be describe...
9      <p>The Ste20-like kinase (SLK) fulfills import...
10     Introduction\r\nSepsis results from a dysregul...
11     <p><span lang="EN-US">With the advent of cell ...
12     Astrocytes form large networks throughout the ...
13     <p><span style="color: #000000"><span style="f...
14     <p>Temporal lobe epilepsy (TLE) with hippocamp...
15     <p>Dendrites of hippocampal CA1 pyramidal cell...
16     <p style="text-align: justify">The specificati...
17     RIMs (Rab3-interacting m

In [52]:
import docx

# open an existing document
doc = docx.Document('test.docx')

# add a table to the end and create a reference variable
# extra row is so we can add the header row
t = doc.add_table(df_abstracts.shape[0]+1, df_abstracts.shape[1])

# add the header rows.
for j in range(df_abstracts.shape[-1]):
    t.cell(0,j).text = df_abstracts.columns[j]

# add the rest of the data frame
for i in range(df_abstracts.shape[0]):
    for j in range(df_abstracts.shape[-1]):
        t.cell(i+1,j).text = str(df_abstracts.values[i,j])

# save the doc
doc.save('test.docx')

PackageNotFoundError: Package not found at 'test.docx'

In [45]:
print(abstract_texts_raw[10])

Introduction
Sepsis results from a dysregulated immune response to infection. It often ends in septic shock, presenting with hypotension, disseminated intravascular coagulation and severe organ damage. Even if an optimal therapy is given, sepsis still shows a high lethality rate of 10-60%. 
The adrenal gland (AG) is a key player for a favourable outcome, as it produces both corticosteroids and catecholamines (CATs). The therapeutic administration of catecholamines is well established, however, the role of glucocorticoids in sepsis is heavily debated, and the the “when” and “how” remains to be investigated.
As the production and release of these hormones can be enhanced via inflammatory cytokines produced by macrophages and dendritic cells, we hypothesized that a directed migration of monocytes specifically into the AG in the early stages of sepsis would be beneficial.
The Cx3CL1-Cx3CR1 axis is known to be important for the directed monocyte migration, thus making it a promising therape

In [5]:
authors_and_institutions = df['allauthors[]']


120

In [39]:
nAbstracts = len(authors_and_institutions)

allInstNames = list()
allAuthNames = list()

for iAbstract in range(nAbstracts):

    fields_aut_inst = authors_and_institutions[iAbstract].split('|')
    num_fields_aut_inst = len(fields_aut_inst)

    name_count = 0
    inst_names = list()
    author_names = list()
    for ifield in range(num_fields_aut_inst):
        #print(name_count, fields_aut_inst[ifield])
        if name_count == 2:
            name_count = 0
            inst_names.append(fields_aut_inst[ifield])
            if fields_aut_inst[ifield-1] in fields_aut_inst[ifield-2]:
                # The last name of author is already in the field for the first name
                author_names.append(fields_aut_inst[ifield-2])
            else:
                # Puting first and last name of author together
                author_names.append(fields_aut_inst[ifield-2]+' '+fields_aut_inst[ifield-1])
        else:
            '''if name_count == 0:
                first_names.append(fields_aut_inst[ifield])
            else:
                last_names.append(fields_aut_inst[ifield])'''
            name_count += 1

    allInstNames.append(inst_names)
    allAuthNames.append(author_names)

    #inst_names, author_names